# CMEMS OSTIA NRT Data Downloader

    Version: 2.0
    Date:    17/09/2019
    Author:  Ben Loveday (Plymouth Marine Laboratory ) and Hayley Evers-King (EUMETSAT)
    Credit:  This code was developed for EUMETSAT under contracts for the Copernicus 
             programme.
    License: This code is offered as open source and free-to-use in the public domain, 
             with no warranty.

**What is this notebook for?**

**What specific tools does this notebook use?**


***

Python is divided into a series of modules that each contain a series of methods for specific tasks. The box below imports all of the moduls we need to complete our plotting task

In [ ]:
# standard tools
import os, sys, json

# specific tools (which can be found here ../Hub_tools/)
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())) + '/Hub_Tools/')
import harmonised_data_access_api_tools as hapi

WEkEO provides access to a huge number of datasets through its 'harmonised-data-access' API. This allows us to query the full data catalogue and download data quickly and directly onto our Jupyter Hub. You can search for what data is available here: https://www.wekeo.eu/dataset-navigator/start.

In order to use the HDA-API we need to provide some authentication credentials, which comes in the form of an api_key. You can get your key from here; https://www.wekeo.eu/api-keys. If you click on the 'show hidden keys' button at the bottom of the page it will reveal a number of keys. The one you need is in the top grey box, and is on the following line:

-H "Authorization: Basic "**YOUR API KEY**"

Replace "YOUR API KEY" below with what you copy from "**YOUR API KEY**" (N.B. you need to keep the quotation marks.)

We will also define a few other parameters including where to download the data to, and if we want the HDA-API functions to be verbose. **Lastly, we will tell the notebook where to find the query we will use to find the data.** These 'JSON' queries are what we use to ask WEkEO for data. They have a very specific form, but allow us quite fine grained control over what data to get. You can find the example one that we will use here: **JSON_templates/RGB/EO_EUM_DAT_SENTINEL-3_OL_1_EFR___.json**

In [ ]:
# your api key:
api_key = "cmJ1UGJQVzZnT09HU2RUWDJhTGFkOGY4RjhnYTpGRmFCTTNoSXluVk1NdEk4b2dPc2ZjMHFOdlVh"
# where the data should be downloaded to:
download_dir_path = "/home/jovyan/work/products"
# where we can find our data query form:
JSON_query_dir = os.path.join(os.getcwd(),'JSON_templates')
# HDA-API loud and noisy?
verbose = False

In [ ]:
# CMEMS OSTIA SST KEY
dataset_id = "EO:MO:DAT:SST_GLO_SST_L4_NRT_OBSERVATIONS_010_001"

In [ ]:
start_dates = []
end_dates = []
for ii in range(2008,2015+1):
    start_dates.append(str(ii)+"-08-01T00:00:00.000Z")
    end_dates.append(str(ii)+"-09-20T00:00:00.000Z")

In [ ]:
# find query file
JSON_query_file = os.path.join(JSON_query_dir,dataset_id.replace(':','_')+".json")
if not os.path.exists(JSON_query_file):
    print('Query file ' + JSON_query_file + ' does not exist')
else:
    print('Found JSON query file for '+dataset_id)

In [ ]:
HAPI_dict = hapi.init(dataset_id, api_key, download_dir_path, verbose=verbose)
HAPI_dict = hapi.get_access_token(HAPI_dict)
HAPI_dict = hapi.accept_TandC(HAPI_dict)

for start_date, end_date in zip(start_dates, end_dates):
    print('Running for: ' + start_date + ' to ' + end_date)

    date_string = start_date.split('T')[0].replace('-','') \
                  + '_' + end_date.split('T')[0].replace('-','')
    
    # load the query
    with open(JSON_query_file, 'r') as f:
        query = f.read()
        query = query.replace("%DATE_START%",start_date)
        query = query.replace("%DATE_END%",end_date)
        query = json.loads(query)
    
    # launch job
    HAPI_dict = hapi.launch_query(HAPI_dict, query)

    # wait for jobs to complete
    HAPI_dict = hapi.check_job_status(HAPI_dict)
    if HAPI_dict['nresults'] == 0:
        print('Nothing to do for this query....')
        continue

    # check results
    HAPI_dict = hapi.get_results_list(HAPI_dict)
    HAPI_dict = hapi.get_download_links(HAPI_dict)

    # check prospective filenames:
    HAPI_dict = hapi.get_filenames(HAPI_dict)
    
    # we ar going to rename files are the downloaded format is not very useful...
    Downloaded_file = HAPI_dict['filenames'][0].split('_')[0] \
                      + '_' + date_string + '.nc'
    print('Will save file as: ' + Downloaded_file)

    # download data
    if os.path.exists(Downloaded_file):
        print('File already exists')
    else:
        HAPI_dict = hapi.download_data(HAPI_dict, skip_existing=True)
        os.rename(HAPI_dict['filenames'][0],Downloaded_file)    